<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/main/SER/CNN/makeSpectrogram_emoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt 
import sklearn as sk

In [ ]:
label = {'W':0, 'L':1, 'E':2, 'A':3, 'F':4, 'T':5, 'N':6} # anger,boredom,disgust,fear,happiness,sadness,neutral

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mel_X = []
mel_Y = []
mfcc_X = []
mfcc_Y = []
root = '/content/drive/MyDrive/team_runner/colab/dataset/emoDB/wave/'

for file in os.listdir(root):
  y, sr = librosa.load(root+file,sr=None)
  y = y[:20400]  # 이미지 하나를 128x128로 맞추기 위함
  if len(y)!=20400: continue  # 그보다 작은 길이의 파일은 생략
  n_fft = int(sr*0.025)
  hop_length = int(sr*0.01)
  
  mel = librosa.power_to_db(librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length))

  mel_X.append(mel)
  
  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=20)
  m = mfcc.min()
  M = mfcc.max()
  print("min",m)
  print("max",M)
  for i in range(len(mfcc)):
    for j in range(len(mfcc[0])): mfcc[i,j]=(mfcc[i,j]-m)/(M-m)
  mfcc_X.append(mfcc)
  
  mel_Y.append(label[file[5]])
  mfcc_Y.append(label[file[5]])

        
mel_X = np.array(mel_X)
mel_Y = np.array(mel_Y)
mfcc_X = np.array(mfcc_X)
mfcc_Y = np.array(mfcc_Y) 


print(mel_X.shape)
print(mfcc_X.shape)
print(mel_Y.shape)
# 데이터 뽑을 때 마다 정규화하는 2차for문이 너무 느리다. 넘파이에는 map같은 함수 없나?

min -708.0041
max 151.99815
min -703.4461
max 209.42188
min -667.77625
max 217.84662
min -680.41205
max 221.62247
min -680.28143
max 135.65063
min -730.2871
max 182.67453
min -673.9257
max 141.97427
min -698.0758
max 164.30571
min -772.9112
max 144.22733
min -708.93274
max 192.68585
min -600.80707
max 208.6927
min -715.45056
max 167.76233
min -676.7186
max 191.44432
min -643.8654
max 174.78656
min -680.19434
max 209.66658
min -760.42773
max 176.35701
min -680.6119
max 180.49568
min -649.4295
max 176.6537
min -659.30835
max 161.27988
min -671.2174
max 181.36537
min -698.1729
max 158.1623
min -605.6358
max 197.41998
min -694.62665
max 189.76962
min -701.59125
max 204.50662
min -710.6545
max 169.83453
min -688.05164
max 169.85056
min -734.8168
max 172.96579
min -685.49243
max 172.54187
min -664.5564
max 194.32028
min -645.52277
max 170.59308
min -649.0311
max 176.77347
min -714.8508
max 149.023
min -775.69934
max 169.35788
min -659.26685
max 198.45839
min -666.9143
max 188.65848
min -659.

In [ ]:
# label별 개수
num = [0]*7
for i in mel_Y: num[i]+=1
for i in range(7): print(['anger','boredom','disgust','fear','happiness','sadness','neutral'][i]+'\t\t'+str(num[i]))

anger		127
boredom		81
disgust		46
fear		68
happiness		71
sadness		62
neutral		79


In [ ]:
shuffle_idx = np.arange(mel_X.shape[0])
np.random.shuffle(shuffle_idx)

mel_X = mel_X[shuffle_idx]
mel_Y = mel_Y[shuffle_idx]
mfcc_X = mfcc_X[shuffle_idx]
mfcc_Y = mfcc_Y[shuffle_idx]

In [ ]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = sk.model_selection.train_test_split(mel_X,mel_Y)
mel_XY = (mel_X_train, mel_X_test, mel_Y_train, mel_Y_test)
np.save("./emo_mel.npy", mel_XY)

mfcc_X_train, mfcc_X_test, mfcc_Y_train, mfcc_Y_test = sk.model_selection.train_test_split(mfcc_X,mfcc_Y)
mfcc_XY = (mfcc_X_train, mfcc_X_test, mfcc_Y_train, mfcc_Y_test)
np.save("./emo_mfcc.npy", mfcc_XY)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
# 1음성->n데이터
scaler = sk.preprocessing.MinMaxScaler()
mel_X = []
mel_Y = []
mfcc_X = []
mfcc_Y = []
root = '/content/drive/MyDrive/team_runner/colab/dataset/emoDB/wave/'
for file in os.listdir(root):
    y, sr = librosa.load(root+file,sr=None)
    y_label = label[file[5]]
    n_fft = int(sr*0.025)
    hop_length = int(sr*0.01)

    for i in range(0,len(y),20400):
      if i+20400>len(y): continue
      yy = y[i:i+20400]  
      
      mel = librosa.power_to_db(librosa.feature.melspectrogram(y=yy, sr=sr, n_fft=n_fft, hop_length=hop_length))
      m = mel.min()
      M = mel.max()
      for i in range(len(mel)):
        for j in range(len(mel[0])): mel[i,j]=(mel[i,j]-m)/(M-m)
      mel_X.append(mel)
      
      mfcc = librosa.feature.mfcc(y=yy, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=20)
      m = mfcc.min()
      M = mfcc.max()
      for i in range(len(mfcc)):
        for j in range(len(mfcc[0])): mfcc[i,j]=(mfcc[i,j]-m)/(M-m)
      mfcc_X.append(mfcc)
      
      mel_Y.append(y_label)
      mfcc_Y.append(y_label)
        
mel_X = np.array(mel_X)
mel_Y = np.array(mel_Y)
mfcc_X = np.array(mfcc_X)
mfcc_Y = np.array(mfcc_Y) 


print(mel_X.shape)
print(mfcc_X.shape)
print(mel_Y.shape)

(911, 128, 128)
(911, 20, 128)
(911,)


In [ ]:
# label별 개수
num = [0]*7
for i in mel_Y: num[i]+=1
for i in range(7): print(['anger','boredom','disgust','fear','happiness','sadness','neutral'][i]+'\t\t'+str(num[i]))

anger		202
boredom		137
disgust		97
fear		93
happiness		106
sadness		167
neutral		109


In [ ]:
shuffle_idx = np.arange(mel_X.shape[0])
np.random.shuffle(shuffle_idx)

mel_X = mel_X[shuffle_idx]
mel_Y = mel_Y[shuffle_idx]
mfcc_X = mfcc_X[shuffle_idx]
mfcc_Y = mfcc_Y[shuffle_idx]

In [ ]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = sk.model_selection.train_test_split(mel_X,mel_Y)
mel_XY = (mel_X_train, mel_X_test, mel_Y_train, mel_Y_test)
np.save("./emo_mel_more.npy", mel_XY)

mfcc_X_train, mfcc_X_test, mfcc_Y_train, mfcc_Y_test = sk.model_selection.train_test_split(mfcc_X,mfcc_Y)
mfcc_XY = (mfcc_X_train, mfcc_X_test, mfcc_Y_train, mfcc_Y_test)
np.save("./emo_mfcc_more.npy", mfcc_XY)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
